# Introduction

LaminDB is an open-source data framework for biology.

```{include} ../README.md
:start-line: 6
:end-line: -4
```

:::{dropdown} LaminDB features

```{include} features-lamindb.md
```
:::

LaminHub is a data collaboration hub built on LaminDB similar to how GitHub is built on git.

:::{dropdown} LaminHub features

```{include} features-laminhub.md
```
:::

Basic features of LaminHub are free. Enterprise features hosted in your or our infrastructure are available on a [paid plan](https://lamin.ai/pricing)!

## Quickstart

```{warning}

Public beta: Close to having converged a stable API, but some breaking changes might still occur.

```

You'll ingest a small dataset while tracking data lineage, and learn how to validate, annotate, query & search.

### Setup

Install the `lamindb` Python package:
```shell
pip install 'lamindb[jupyter,bionty]'
```

Initialize a LaminDB instance mounting plug-in {py:mod}`bionty` for biological types:

In [ ]:
import lamindb as ln

# artifacts are stored in a local directory `./lamin-intro`
ln.setup.init(schema="bionty", storage="./lamin-intro")

# tag your code with auto-generated identifiers for a script or notebook
ln.settings.transform.stem_uid = "FPnfDtJz8qbE"
ln.settings.transform.version = "1"

# track the execution of a transform with a global run context
ln.track()

### Manage artifacts

In [ ]:
import pandas as pd

# dummy data
df = pd.DataFrame(
    {"CD8A": [1, 2, 3], "CD4": [3, 4, 5], "CD14": [5, 6, 7], "perturbation": ["DMSO", "IFNG", "DMSO"]},
    index=["observation1", "observation2", "observation3"],
)

With {class}`~lamindb.Artifact`, you can manage data batches & models in storage as files, folders or arrays.

In [ ]:
artifact = ln.Artifact.from_df(df, description="my RNA-seq", version="1")

Any artifact comes with typed, relational metadata:

In [ ]:
artifact.describe()

If you save an artifact, you'll save data & metadata in one operation:

In [ ]:
artifact.save()

For any artifact, you can view its data lineage:

In [ ]:
artifact.view_lineage()

:::{dropdown} Data provenance in the UI

The screenshot shows a notebook with its latest report, runs, output files, and parent notebooks. On the run view, you'll see input files.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8aBoM.png" width="700px">

:::

Loading an artifact returns an object determined by its `.accessor` and `.suffix`:

In [ ]:
artifact.load()

### Query

A simple query:

In [ ]:
ln.Artifact.filter(description="my RNA-seq").df()

To query all artifacts ingested from a notebook with title `"Introduction"`:

In [ ]:
ln.Transform.filter(name="Introduction").df()

In [ ]:
transform = ln.Transform.filter(name="Introduction", version="1").one()
artifacts = ln.Artifact.filter(transform=transform).all()

Because, under-the-hood, LaminDB is SQL & Django, you can write arbitrarily complex relational queries:

In [ ]:
artifacts = ln.Artifact.filter(transform__name__icontains="intro", created_by__handle="anonymous").all()

:::{dropdown} Query in the UI

If you work with a remote instance on LaminHub, you can compose queries as shown below.

Because LaminDB's metadata-management is based on SQL, registries can easily have 10s of millions of rows.

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/L188T2JjzZHWHfv2S0ib.png" width="700px">

:::

### Search

Search the {class}`~lamindb.Artifact` registry:

In [ ]:
ln.Artifact.search("RNAseq")

Or search any other registry, e.g., {class}`~lamindb.Transform`:

In [ ]:
ln.Transform.search("intro")

### Look up

We can look up records in any registry with auto-complete until we have more than 200k entries:

In [ ]:
users = ln.User.lookup()

:::{dropdown} Show me a screenshot

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/lgRNHNtMxjU0y8nIagt7.png" width="400px">

:::

## Features & labels

For instance, populate the feature registry ({class}`~lamindb.Feature`) based on the columns measured in the `DataFrame`:

In [ ]:
features = ln.Feature.from_df(df)
ln.save(features)

The registry now looks like this:

In [ ]:
ln.Feature.df()

Let's also create a label using {class}`~lamindb.ULabel`, LaminDB's universal label registry.

(Later, we'll use typed labels to deal with, e.g., 100k gene identifiers.)

In [ ]:
study = ln.ULabel(name="Candidate marker study")
study.save()
ln.ULabel.df()

We can model hierachical labels like so:

In [ ]:
is_study= ln.ULabel(name="is_study")
is_study.save()
is_study.children.add(study)
study.view_parents()

Let us do the same for perturbation labels: 

In [ ]:
perturbations = [ln.ULabel(name=label) for label in df["perturbation"].unique()]
ln.save(perturbations)
is_perturbation = ln.ULabel(name="is_perturbation")
is_perturbation.save()
is_perturbation.children.add(*perturbations)
is_perturbation.view_parents(with_children=True)

## Validate & annotate

Now that we defined features, we can link them to the artifact:

In [ ]:
artifact.features.add(features)

In [ ]:
artifact.describe()

Annotating an artifact with a label works like so:

In [ ]:
artifact.labels.add(study)
artifact.describe()

We can also associate labels with a feature:

In [ ]:
features_lookup = ln.Feature.lookup()
artifact.labels.add(perturbations, feature=features_lookup.perturbation)
artifact.describe()

:::{dropdown} Artifacts with context in the UI

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/DjVOPEBiAcGlt3Gq7Qc1.png" width="700px">

:::

Get lookup object for the entities of interest:

In [ ]:
studies = is_study.children.lookup()
perturbations = is_perturbation.children.lookup()

In [ ]:
ln.Artifact.filter(ulabels=studies.candidate_marker_study).filter(ulabels=perturbations.ifng).one()

Delete an artifact:

In [ ]:
artifact.delete(permanent=True)

## Biological types

{class}`~lamindb.Feature` and {class}`~lamindb.ULabel` will get you pretty far.

But if you use an entity many times, you typically want a dedicated registry.

Let's do this with {class}`~bionty.Gene` and {class}`~bionty.Tissue` from plug-in {py:mod}`bionty`:

### Access public ontologies 

Import gene records from a public ontology, which we'll use to validate features:

In [ ]:
import bionty as bt

genes = bt.Gene.from_values(df.columns, organism="human")
ln.save(genes)
bt.Gene.df()

### Validate typed features

To manage features of different types, let us use an `AnnData` object, which comes with two slots for storing data & metadata:

In [ ]:
import anndata as ad

adata = ad.AnnData(df[["CD8A", "CD4", "CD14"]], obs=df[["perturbation"]])

Create an artifact & validate features using the symbol field of `Gene`:

In [ ]:
artifact = ln.Artifact.from_anndata(adata, description="my RNA-seq")
artifact.save()

### Annotate with typed labels

Search the public tissue ontology from the bionty store:

In [ ]:
bt.Tissue.public().search("umbilical blood").head(2)

Define a tissue label:

In [ ]:
tissue = bt.Tissue.from_public(name="umbilical cord blood")
tissue.save()
tissue.view_parents(distance=2)

Annotate the artifact:

In [ ]:
artifact.features.add_from_anndata(var_field=bt.Gene.symbol, organism="human")
artifact.labels.add(study)
artifact.labels.add(adata.obs.perturbation, feature=features_lookup.perturbation)
artifact.labels.add(tissue)
artifact.describe()

Query for genes & the linked artifacts:

In [ ]:
genes = bt.Gene.filter(organism__name="human").lookup()

# all gene sets measuring CD8A
genesets_with_cd8a = ln.FeatureSet.filter(genes=genes.cd8a).all()

# all artifacts measuring CD8A
ln.Artifact.filter(feature_sets__in=genesets_with_cd8a).df()

### Manage biological registries

Create a cell type registry and add a new cell state (from [here](bio-registries)):

In [ ]:
# create an ontology-coupled cell type record and save it
bt.CellType.from_public(name="neuron").save()

# create a record to track a new cell state
new_cell_state = bt.CellType(name="my neuron cell state", description="explains X")
new_cell_state.save()

# express that it's a neuron state
cell_types = bt.CellType.lookup()
new_cell_state.parents.add(cell_types.neuron)

# view ontological hierarchy
new_cell_state.view_parents(distance=2)

## Collections of artifacts

Assume we now run a pipeline in which we access a new batch of data:

In [ ]:
transform = ln.Transform(name="Cell Ranger", type="pipeline", version="1")
ln.track(transform=transform)

Access a new batch of data:

In [ ]:
df = pd.DataFrame(
    {
        "CD8A": [2, 3, 3],
        "CD4": [3, 4, 5],
        "CD38": [4, 2, 3],
        "perturbation": ["DMSO", "IFNG", "IFNG"]
    },
    index=["observation4", "observation5", "observation6"],
)
adata = ad.AnnData(df[["CD8A", "CD4", "CD38"]], obs=df[["perturbation"]])

Because gene `"CD38"` is not yet registered, it doesn't yet validate:

In [ ]:
bt.Gene.validate(adata.var_names, field=bt.Gene.symbol, organism="human");

Let's add it to the `Gene` registry and link to the artifact - now all features validate:

In [ ]:
bt.Gene.from_public(symbol="CD38", organism="human").save()
artifact2 = ln.Artifact.from_anndata(
    adata, description="my RNA-seq batch 2"
)
artifact2.save()
artifact2.features.add_from_anndata(var_field=bt.Gene.symbol, organism="human")
artifact2.describe()

Create a collection using {class}`~lamindb.Collection`:

In [ ]:
collection = ln.Collection([artifact, artifact2], name="my RNA-seq collection", version="1")
collection.save()
collection.describe()
collection.view_lineage()

If it's small enough, you can load the entire collection into memory as if it was one:

In [ ]:
collection.load()

Iterate over its artifacts:

In [ ]:
collection.artifacts.df()

## Save notebooks & scripts

If you call {func}`~lamindb.finish()`, you save the run report, source code, and compute environment to your default storage location.

In [ ]:
ln.finish()

See an example for this introductory notebook [here](https://lamin.ai/laminlabs/lamindata/transform/FPnfDtJz8qbE5zKv).

:::{dropdown} Show me a screenshot

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/RGXj5wcAf7EAc6J8aBoM.png" width="700px">

:::

If you want to download a notebook or script, call:

```bash
lamin stage https://lamin.ai/laminlabs/lamindata/transform/FPnfDtJz8qbE5zKv
```


## Machine learning models

Using {class}`~lamindb.core.MappedCollection` you can train machine learning models on large collections of artifacts:

```
from torch.utils.data import DataLoader, WeightedRandomSampler
dataset = collection.mapped(label_keys=["perturbation"])
sampler = WeightedRandomSampler(
    weights=dataset.get_label_weights("perturbation"), num_samples=len(dataset)
)
dl = DataLoader(dataset, batch_size=2, sampler=sampler)
for batch in dl:
    pass
```

## Data lineage

View the sequence of data transformations ({class}`~lamindb.Transform`) in a project (from [here](docs:project-flow), based on [Schmidt _et al._, 2022](https://pubmed.ncbi.nlm.nih.gov/35113687/)):

```python
transform.view_parents()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/b0geN1HDHXlORqMOOPay.svg" width="400">

Or, the generating flow of an artifact:

```python
artifact.view_lineage()
```

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/KQmzmmLOeBN0C8Ykitjn.svg" width="800">


Both figures are based on mere calls to `ln.track()` in notebooks, pipelines & app.

## Mesh of databases

LaminDB is a distributed system like git. Similar to cloning a repository, collaborators can load your instance on the command-line using:

```shell
lamin load myhandle/myinstance
```

## Custom schemas

LaminDB can be customized & extended with schema & app plug-ins building on the [Django](https://github.com/django/django) ecosystem. Examples are

- [bionty](./bionty): Registries for basic biological entities, coupled to public ontologies.
- [wetlab](https://github.com/laminlabs/wetlab): Exemplary custom schema to manage samples, treatments, etc. 

If you'd like to create your own schema or app:

1. Create a git repository with registries similar to [wetlab](https://github.com/laminlabs/wetlab)
2. Create & deploy migrations via `lamin migrate create` and `lamin migrate deploy`

It's fastest if we do this for you based on our templates within an enterprise plan.

## Design

### Why?

See this [blog post](https://lamin.ai/blog/2022/problems).

### Schema & API

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/XoTQFCmmj2uU4d2xyj9t.png" width="350px" style="background: transparent" align="right">

LaminDB provides a SQL schema for common entities: {class}`~lamindb.Artifact`, {class}`~lamindb.Collection`, {class}`~lamindb.Transform`, {class}`~lamindb.Feature`, {class}`~lamindb.ULabel` etc. - see the [API reference](reference) or the [source code](https://github.com/laminlabs/lnschema-core/blob/main/lnschema_core/models.py).

The core schema is extendable through plug ins (see blue vs. red entities in **graphic**), e.g., with basic biological ({class}`~bionty.Gene`, {class}`~bionty.Protein`, {class}`~bionty.CellLine`, etc.) & operational entities (`Biosample`, `Techsample`, `Treatment`, etc.).

```{dropdown} What is the schema language?

Data models are defined in Python using the Django ORM. Django translates them to SQL tables.

[Django](https://github.com/django/django) is one of the most-used & highly-starred projects on GitHub (~1M dependents, ~73k stars) and has been robustly maintained for 15 years.

In the first year, LaminDB used SQLModel/SQLAlchemy -- we might bring back compatibility.

```

On top of the schema, LaminDB is a Python API that abstracts over storage & database access, data transformations, and (biological) ontologies.

The code for this is open-source & accessible through the dependencies & repositories listed below.
 
### Dependencies

- Data is stored in a platform-independent way: 
    - location → local, on AWS S3 or GCP Storage, accessed through `fsspec`
    - format → blob-like artifacts or queryable formats like parquet, zarr, HDF5, TileDB, ...
- Metadata is stored in SQL: current backends are SQLite (small teams) and Postgres (any team size).
- Django ORM for schema management & metadata queries.
- Biological knowledge sources & ontologies: see [Bionty](./bionty).

For more details, see the [pyproject.toml](https://github.com/laminlabs/lamindb/blob/main/pyproject.toml) artifact in lamindb & the linked repositories below.

### Repositories

LaminDB and its plug-ins consist in open-source Python libraries & publicly hosted metadata assets:

- [lamindb](https://github.com/laminlabs/lamindb): Core API, which builds on the [core schema](https://github.com/laminlabs/lnschema-core).
- [bionty](https://github.com/laminlabs/bionty): Registries for basic biological entities, coupled to public ontologies.
- [wetlab](https://github.com/laminlabs/wetlab): Exemplary custom schema to manage samples, treatments, etc.
- [lamindb-setup](https://github.com/laminlabs/lamindb-setup): Setup & configure LaminDB, client for LaminHub.
- [lamin-cli](https://github.com/laminlabs/lamin-cli): CLI for `lamindb` and `lamindb-setup`.
- [nbproject](https://github.com/laminlabs/nbproject): Metadata parser for Jupyter notebooks.
- [lamin-utils](https://github.com/laminlabs/lamin-utils): Generic utilities, e.g., a logger.
- [readfcs](https://github.com/laminlabs/readfcs): FCS artifact reader.
<!-- [bionty-assets](https://github.com/laminlabs/bionty-assets): Hosted assets of parsed public biological ontologies. -->

LaminHub is not open-sourced, and neither are plug-ins that model lab operations.


### Assumptions & principles

<img src="https://lamin-site-assets.s3.amazonaws.com/.lamindb/BunYmHkyFLITlM5MYQci.svg" width="350px" style="background: transparent" align="right">

1. Data is generated in batches by instruments that process physical samples.
2. Batches are transformed into more useful representations
3. Semantics of high-level embeddings ("inflammatory", "lipophile") are anchored in experimental metadata and knowledge (ontologies)
4. Experimental metadata is another ontology type
5. Experiments measure features ({class}`~lamindb.Feature`, {class}`~bionty.CellMarker`, ...)
6. Samples are annotated by labels ({class}`~lamindb.ULabel`, {class}`~bionty.CellLine`, ...)
7. Learning and data warehousing both iterate transformations (see **graphic**, {class}`~lamindb.Transform`)
8. Basic biological entities should have the same meaning to anyone and across any data platform

### Influences

LaminDB was influenced by many other projects, see {doc}`docs:influences`.

## Notebooks

- Find all tutorial & guide notebooks [here](https://github.com/laminlabs/lamindb/tree/main/docs/) and use cases [here](https://github.com/laminlabs/lamin-usecases).
- You can run these notebooks in hosted versions of JupyterLab, e.g., [Saturn Cloud](https://github.com/laminlabs/run-lamin-on-saturn), Google Vertex AI, Google Colab, and others.

In [ ]:
# clean up test instance
!lamin delete --force lamin-intro
!rm -r lamin-intro